In [3]:
import os
import cv2
import mediapipe as mp
import numpy as np
import time
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageDraw
from PIL import ImageFont

# ---------------- Konfigurasi & Gaya ----------------
ONNX_MODEL_PATH = "ASL_Mediapipe.onnx"
INPUT_DIM = 63
CLASSES = list("ABCDEFGHIJKLMNOPQRSTUVWXYZ") + ['nothing', 'space', 'del']

SHOW_LANDMARKS = False

ROI_SIZE = (220, 220)       # ukuran tampilan ROI (px)
EXAMPLE_SIZE = (280, 280)   # ukuran tampilan contoh (px)
WINDOW_GEOMETRY = "1280x820"

# # --- Palet Warna (Flat UI Inspired) ---
# BG_DARK = "#2c3e50"         # Dark Navy/Slate Background
# BG_MID = "#34495e"          # Slightly Lighter Frame Background
# TEXT_LIGHT = "#ecf0f1"      # Light Text (Almost White)
# ACCENT_PRIMARY = "#2ecc71"    # Save Button / Prediction Accent
# ACCENT_SECONDARY = "#e74c3c"      # Quit Button
# ACCENT_SECONDARY = "#f39c12"   # Clear Button
# ACCENT_MUTED = "#3498db"     # Bounding Box Color

# === Warna Tema (Black + Yellow Theme) ===
BG_DARK = "#1E2328"          # Background utama (gelap)
BG_MID = "#2A2E34"           # Frame sekunder
TEXT_LIGHT = "#FED053"       # Warna teks
ACCENT_PRIMARY = "#F5B301"   # Tombol utama / aksen terang
ACCENT_SECONDARY = "#FED053" # Tombol sekunder
ACCENT_MUTED = "#3B3F46"     # Warna tombol pasif

# --- Konfigurasi Efek Flash ---
FLASH_DURATION_FRAMES = 5
FLASH_ALPHA = 0.4

# ---------------- MediaPipe Hands ----------------
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# ---------------- Load ONNX ----------------
try:
    net = cv2.dnn.readNetFromONNX(ONNX_MODEL_PATH)
    print("Model ONNX berhasil dimuat.")
except Exception as e:
    print(f"WARNING: gagal load ONNX: {e}")
    net = None

# ---------------- Fungsi BBox ----------------
def get_bbox(hand_landmarks, frame_w, frame_h, margin=40):
    xs = [lm.x * frame_w for lm in hand_landmarks.landmark]
    ys = [lm.y * frame_h for lm in hand_landmarks.landmark]
    x_min, x_max = int(min(xs)) - margin, int(max(xs)) + margin
    y_min, y_max = int(min(ys)) - margin, int(max(ys)) + margin
    x_min, y_min = max(0, x_min), max(0, y_min)
    x_max, y_max = min(frame_w - 1, x_max), min(frame_h - 1, y_max)
    return x_min, y_min, x_max, y_max

def draw_fancy_bbox(img, x1, y1, x2, y2, color=ACCENT_MUTED, thickness=2, corner_len=30):
    # Menggunakan warna yang lebih cerah
    color_bgr = (color[2], color[1], color[0]) # Konversi hex string ke BGR tuple
    cv2.rectangle(img, (x1, y1), (x2, y2), color_bgr, thickness)
    # corner accents
    cv2.line(img, (x1, y1), (x1 + corner_len, y1), color_bgr, thickness)
    cv2.line(img, (x1, y1), (x1, y1 + corner_len), color_bgr, thickness)
    cv2.line(img, (x2, y1), (x2 - corner_len, y1), color_bgr, thickness)
    cv2.line(img, (x2, y1), (x2, y1 + corner_len), color_bgr, thickness)
    cv2.line(img, (x1, y2), (x1 + corner_len, y2), color_bgr, thickness)
    cv2.line(img, (x1, y2), (x1, y2 - corner_len), color_bgr, thickness)
    cv2.line(img, (x2, y2), (x2 - corner_len, y2), color_bgr, thickness)
    cv2.line(img, (x2, y2), (x2, y2 - corner_len), color_bgr, thickness)

# Fungsi helper untuk konversi hex ke BGR (untuk OpenCV)
def hex_to_bgr(hex_color):
    h = hex_color.lstrip('#')
    rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
    return (rgb[2], rgb[1], rgb[0]) # OpenCV uses BGR

# ---------------- Tkinter UI ----------------
class ASLApp:
    def __init__(self, root):
        self.root = root
        self.root.title("ASL to Text Translator")
        self.root.geometry(WINDOW_GEOMETRY)
        self.root.config(bg=BG_DARK) # Dark background for root
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=3)
        self.root.grid_columnconfigure(1, weight=1)

        # Left frame = video
        self.left_frame = tk.Frame(root, bg=BG_DARK)
        self.left_frame.grid(row=0, column=0, sticky="nsew", padx=15, pady=15)
        self.left_frame.grid_rowconfigure(0, weight=1)
        self.left_frame.grid_columnconfigure(0, weight=1)

        # Right frame = ROI + example
        self.right_frame = tk.Frame(root, bg=BG_DARK)
        self.right_frame.grid(row=0, column=1, sticky="nsew", padx=(0,15), pady=15)
        self.right_frame.grid_rowconfigure(0, weight=0)
        self.right_frame.grid_rowconfigure(1, weight=1)
        self.right_frame.grid_columnconfigure(0, weight=1)

        # Video label (left) - Darker background for the video area
        self.video_label = tk.Label(self.left_frame, bd=0, relief="flat", bg=BG_MID)
        self.video_label.grid(row=0, column=0, sticky="nsew")

        # Sidebar: first row = ROI and prediction text
        top_side = tk.Frame(self.right_frame, bg=BG_DARK)
        top_side.grid(row=0, column=0, sticky="n", pady=(0,15))

        # Captured ROI label (with a raised border for definition)
        self.ss_label = tk.Label(top_side, bd=3, relief="groove", bg="#555555")
        self.ss_label.pack(padx=10, pady=10)
        self._create_placeholder_roi()

        # Prediction label (large, bold, and accented)
        self.pred_label = tk.Label(
            top_side, 
            text="Prediction: -", 
            font=("Arial", 14, "bold"),
            fg=ACCENT_PRIMARY, # Use green accent color
            bg=BG_DARK
        )
        self.pred_label.pack(pady=(5,15))

        # Sidebar: second row = example image
        bottom_side = tk.Frame(self.right_frame, bg=BG_DARK)
        bottom_side.grid(row=1, column=0, sticky="n")

        self.example_label = tk.Label(bottom_side, bd=3, relief="groove", bg="#555555")
        self.example_label.pack(padx=10, pady=10)
        self._create_placeholder_example()
        
        # Label Title for Example Image
        tk.Label(
            bottom_side, 
            text="Tangan Contoh (Referensi)", 
            font=("Arial", 10), 
            fg=TEXT_LIGHT, 
            bg=BG_DARK
        ).pack(pady=(0, 5))


        # Text area (bottom across full window) - Larger, darker, more legible
        self.text_area = tk.Text(
            root, 
            height=4, 
            font=("Arial", 18), 
            bg=BG_MID, 
            fg=TEXT_LIGHT, 
            insertbackground=TEXT_LIGHT, # Cursor color
            bd=5, 
            relief="flat", 
            padx=10, 
            pady=10
        )
        self.text_area.grid(row=1, column=0, columnspan=2, sticky="ew", padx=15, pady=(0,10))
        
        # Frame for buttons
        btn_frame = tk.Frame(root, bg=BG_DARK)
        btn_frame.grid(row=2, column=0, columnspan=2, sticky="ew", padx=15, pady=(0,15))
        btn_frame.grid_columnconfigure((0,1,2), weight=1)

        # Styled Buttons
        button_style = {
            'fg': ACCENT_MUTED, 
            'relief': 'flat', 
            'font': ("Arial", 12, "bold"), 
            'height': 2
        }

        tk.Button(
            btn_frame, 
            text="Clear All", 
            command=self.clear_text, 
            bg=ACCENT_SECONDARY, 
            **button_style
        ).grid(row=0, column=0, sticky="ew", padx=(0, 5))
        
        tk.Button(
            btn_frame, 
            text="Save to Text File", 
            command=self.save_text, 
            bg=ACCENT_PRIMARY, 
            **button_style
        ).grid(row=0, column=1, sticky="ew", padx=5)
        
        tk.Button(
            btn_frame, 
            text="Quit", 
            command=self.quit_app, 
            bg=ACCENT_SECONDARY, 
            **button_style
        ).grid(row=0, column=2, sticky="ew", padx=(5, 0))

        # kamera
        self.cap = cv2.VideoCapture(0)
        if not self.cap.isOpened():
            print("ERROR: Tidak dapat membuka kamera.")
            self.root.destroy()
            return

        # prediksi vars
        self.last_capture_time = time.time()
        self.interval = 3
        self.sentence = ""
        self.prediction_text = "No Hand Detected"
        self.last_pred_class = None

        # timer deteksi tangan
        self.hand_detected_time = None
        self.wait_time_before_ss = 2

        # --- [BARU] Variabel untuk kontrol efek flash ---
        self.flash_timer = 0
        self.last_bbox = None

        # coba load example (ganti nama file jika perlu)
        self.load_example_image("gambar_tangan.jpg")

        # start
        self.update_frame()

    def _create_placeholder_roi(self):
        # Placeholder untuk ROI yang lebih gelap
        img = Image.new("RGB", ROI_SIZE, (40, 40, 40))
        draw = ImageDraw.Draw(img)
        # Mencoba menggunakan font default yang tersedia
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except IOError:
            font = ImageFont.load_default()
            
        txt = "Captured ROI"
        bbox = draw.textbbox((0, 0), txt, font=font)
        w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        draw.text(((ROI_SIZE[0]-w)//2, (ROI_SIZE[1]-h)//2), txt, fill=(180,180,180), font=font)
        self.ss_tk = ImageTk.PhotoImage(img)
        self.ss_label.configure(image=self.ss_tk)
        self.ss_label.image = self.ss_tk

    def _create_placeholder_example(self):
        # Placeholder untuk Example yang lebih gelap
        img = Image.new("RGB", EXAMPLE_SIZE, (40, 40, 40))
        draw = ImageDraw.Draw(img)
        try:
            font = ImageFont.truetype("arial.ttf", 16)
        except IOError:
            font = ImageFont.load_default()
            
        txt = "Example Image"
        bbox = draw.textbbox((0, 0), txt, font=font)
        w, h = bbox[2] - bbox[0], bbox[3] - bbox[1]
        draw.text(((EXAMPLE_SIZE[0]-w)//2, (EXAMPLE_SIZE[1]-h)//2), txt, fill=(180,180,180), font=font)
        self.example_tk = ImageTk.PhotoImage(img)
        self.example_label.configure(image=self.example_tk)
        self.example_label.image = self.example_tk

    def load_example_image(self, path):
        paths_to_try = [path, os.path.join(os.getcwd(), path)]
        img = None
        for p in paths_to_try:
            try:
                # Perlu diingat, file 'gambar_tangan.jpg' harus ada di direktori yang benar
                img = Image.open(p)
                break
            except Exception:
                img = None
        if img is None:
            print("WARNING: example image not found. Using placeholder.")
            return
        img = img.convert("RGB").resize(EXAMPLE_SIZE)
        self.example_tk = ImageTk.PhotoImage(img)
        self.example_label.configure(image=self.example_tk, text="")
        self.example_label.image = self.example_tk

    def update_frame(self):
        ret, frame = self.cap.read()
        if not ret:
            self.root.after(10, self.update_frame)
            return

        frame = cv2.flip(frame, 1)
        h, w, _ = frame.shape

        # --- Logika untuk menggambar efek flash ---
        if self.flash_timer > 0 and self.last_bbox:
            x1, y1, x2, y2 = self.last_bbox
            # Buat layer overlay putih
            overlay = frame.copy()
            cv2.rectangle(overlay, (x1, y1), (x2, y2), (255, 255, 255), -1)
            # Gabungkan frame asli dengan overlay
            frame = cv2.addWeighted(overlay, FLASH_ALPHA, frame, 1 - FLASH_ALPHA, 0)
            
            self.flash_timer -= 1
            if self.flash_timer == 0:
                self.last_bbox = None

        image_rgb = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            hand_landmarks = results.multi_hand_landmarks[0]
            x1, y1, x2, y2 = get_bbox(hand_landmarks, w, h)
            # Menggunakan warna ACCENT_MUTED
            draw_fancy_bbox(frame, x1, y1, x2, y2, color=hex_to_bgr(ACCENT_MUTED), thickness=2)

            if SHOW_LANDMARKS:
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            current_time = time.time()
            if self.hand_detected_time is None:
                self.hand_detected_time = current_time

            # hanya SS kalau sudah > 2 detik tangan terlihat
            if current_time - self.hand_detected_time >= self.wait_time_before_ss:
                if current_time - self.last_capture_time >= self.interval:
                    
                    # --- Memicu efek flash saat capture ---
                    self.flash_timer = FLASH_DURATION_FRAMES
                    self.last_bbox = (x1, y1, x2, y2)
                    
                    # crop safely
                    x1c, y1c, x2c, y2c = max(0,int(x1)), max(0,int(y1)), min(w-1,int(x2)), min(h-1,int(y2))
                    if x2c > x1c and y2c > y1c:
                        hand_crop = frame[y1c:y2c, x1c:x2c]
                        if hand_crop.size > 0:
                            roi_resized = cv2.resize(hand_crop, ROI_SIZE)
                            roi_pil = Image.fromarray(cv2.cvtColor(roi_resized, cv2.COLOR_BGR2RGB))
                            self.ss_tk = ImageTk.PhotoImage(roi_pil)
                            self.ss_label.configure(image=self.ss_tk, text="")
                            self.ss_label.image = self.ss_tk

                    # prediksi
                    if net is not None:
                        input_data = []
                        for lm in hand_landmarks.landmark:
                            input_data.extend([lm.x, lm.y, lm.z])
                        input_data = np.array(input_data, dtype=np.float32).reshape(1, INPUT_DIM)
                        net.setInput(input_data)
                        output = net.forward()
                        class_idx = int(np.argmax(output[0]))
                        confidence = float(output[0][class_idx])
                        pred_class = CLASSES[class_idx] if class_idx < len(CLASSES) else str(class_idx)
                        self.prediction_text = f"{pred_class} ({confidence*100:.1f}%)"

                        accept = False
                        if pred_class == "space" and confidence > 0.5:
                            accept = True
                        elif pred_class == "del" and confidence > 0.5:
                            accept = True
                        elif pred_class not in ["nothing", "space", "del"] and confidence > 0.5:
                            accept = True

                        if accept and pred_class != self.last_pred_class:
                            if pred_class == "space":
                                self.sentence += " "
                            elif pred_class == "del":
                                self.sentence = self.sentence[:-1]
                            else:
                                self.sentence += pred_class
                            self.text_area.delete("1.0", tk.END)
                            self.text_area.insert("1.0", self.sentence)
                            self.last_pred_class = pred_class

                        self.pred_label.configure(text=f"Prediction: {self.prediction_text}")

                    self.last_capture_time = current_time
        else:
            self.last_pred_class = None
            self.hand_detected_time = None

        # display webcam frame
        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        imgtk = ImageTk.PhotoImage(image=img)
        self.video_label.imgtk = imgtk
        self.video_label.configure(image=imgtk)

        self.root.after(10, self.update_frame)

    def clear_text(self):
        self.text_area.delete("1.0", tk.END)
        self.sentence = ""
        self.last_pred_class = None

    def save_text(self):
        text = self.text_area.get("1.0", tk.END).strip()
        file = filedialog.asksaveasfilename(
            defaultextension=".txt", 
            filetypes=[("Text files", "*.txt")]
        )
        if file:
            try:
                with open(file, "w", encoding="utf-8") as f:
                    f.write(text)
            except Exception as e:
                print(f"Error saving file: {e}")

    def quit_app(self):
        if self.cap and self.cap.isOpened():
            self.cap.release()
        self.root.destroy()

# ---------------- Main ----------------
if __name__ == "__main__":
    # Pindahkan helper untuk konversi warna ke luar class jika digunakan di luar
    def hex_to_bgr(hex_color):
        h = hex_color.lstrip('#')
        rgb = tuple(int(h[i:i+2], 16) for i in (0, 2, 4))
        return (rgb[2], rgb[1], rgb[0]) # OpenCV uses BGR

    root = tk.Tk()
    app = ASLApp(root)
    root.mainloop()


Model ONNX berhasil dimuat.
